In [1]:
import matplotlib.pyplot as plt
from astropy import units as u
from poliastro.bodies import Moon, Earth, Sun
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
from poliastro.twobody.propagation import cowell as cowell
from poliastro.twobody.propagation import farnocchia as farnocchia
from poliastro.twobody.propagation import vallado as vallado
from poliastro.twobody.propagation import mikkola as mikkola
from poliastro.twobody.propagation import markley as markley
from poliastro.twobody.propagation import pimienta as pimienta
from poliastro.twobody.propagation import gooding as gooding
from poliastro.twobody.propagation import danby as danby
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.perturbations import J3_perturbation
from poliastro.core.perturbations import third_body
from poliastro.ephem import Ephem
import pandas as pd
from astropy import time
from utility.utils import *
%matplotlib tk
plt.style.use("seaborn-paper")

Moon_J2= 2.0323e-4
Moon_J3 = 8.4759e-6

epoch = time.Time("2020-11-15 11:00:00").tdb  # UTC by default

In [2]:
earth = Ephem.from_body(Earth, epoch)
# Orbit.from_ephem(Sun, earth, epoch)

In [3]:
# Orbit.from_body_ephem(Sun, earth)

In [17]:
############################################
dt = 10
############################################
# df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
df = pd.read_csv(f'DATA/ephemeris sat/Ephemeris HPOP stepsize=10.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

# Initial State Vector:
x, y, z = real_X[0], real_Y[0], real_Z[0]
vx, vy, vz = real_Vxs[0], real_Vys[0], real_Vzs[0]

In [18]:
r = np.array([x,y,z])*u.km
v = np.array([vx,vy,vz])*u.km/u.s
print(r,v)

[-1512.69930704  -952.12371386     0.        ] km [ 0.882232 -1.401658 -0.      ] km / s


In [22]:
orbit = Orbit.from_vectors(Moon, r, v, epoch=epoch)

In [35]:
orbit.plot()
plt.show()

In [38]:
frame = OrbitPlotter3D()
frame.plot(orbit)

In [6]:
def propagate_noJ2(r,v,t):
    r = r * u.km
    v = v * u.km / u.s
    initial = Orbit.from_vectors(Moon, r, v, epoch=epoch)
    final = initial.propagate(t*u.s, method=cowell)
    return final.r.to_value(), final.v.to_value()

def propagate_J2(r,v,t):
    r = r * u.km
    v = v * u.km / u.s
    initial = Orbit.from_vectors(Moon, r, v)
    final = initial.propagate(t*u.s, method=cowell, ad=J2_perturbation, J2=Moon_J2, R=Moon.R.to(u.km).value)

    return final.r.to_value(), final.v.to_value()

def propagate_J3(r,v,t):
    r = r * u.km
    v = v * u.km / u.s
    initial = Orbit.from_vectors(Moon, r, v)
    final = initial.propagate(t*u.s, method=cowell, ad=J3_perturbation, J3=Moon_J3, R=Moon.R.to(u.km).value)

    return final.r.to_value(), final.v.to_value()

def accel(t0, state, k):
    J2 = J2_perturbation(t0, state, Moon.k.to(u.km**3/u.s**2), Moon_J2, Moon.R.to(u.km).value)
    J3 = J3_perturbation(t0, state, Moon.k.to(u.km**3/u.s**2), Moon_J3, Moon.R.to(u.km).value)
    return (J2+J3)


def propagate_J2J3(r,v):
    r = r * u.km
    v = v * u.km / u.s
    initial = Orbit.from_vectors(Moon, r, v)

    rr , vv = r.to_value().T, v.to_value().T
    
    final = initial.propagate(10*u.s, method=cowell, ad = accel)
    return final.r.to_value(), final.v.to_value()

def accel_TB(t0, state, k):
    J2 = J2_perturbation(t0, state, Moon.k.to(u.km**3/u.s**2), Moon_J2, Moon.R.to(u.km).value)
    J3 = J3_perturbation(t0, state, Moon.k.to(u.km**3/u.s**2), Moon_J3, Moon.R.to(u.km).value)
    TB = third_body(t0, state, Moon.k.to(u.km**3/u.s**2), Earth.k.to(u.km**3/u.s**2), ss)

    return (J2+J3+TB)


def propagate_J2J3TB(r,v):
    r = r * u.km
    v = v * u.km / u.s
    initial = Orbit.from_vectors(Moon, r, v)

    rr , vv = r.to_value().T, v.to_value().T
    
    final = initial.propagate(10*u.s, method=cowell, ad = accel_TB)
    return final.r.to_value(), final.v.to_value()

In [7]:
r = [real_X[0], real_Y[0], real_Z[0]]
v = [real_Vxs[0], real_Vys[0], real_Vzs[0]]  

X1, Y1, Z1 = [r[0]], [r[1]], [r[2]]
X2, Y2, Z2 = [r[0]], [r[1]], [r[2]]
X3, Y3, Z3 = [r[0]], [r[1]], [r[2]]
X4, Y4, Z4 = [r[0]], [r[1]], [r[2]]

r1, r2, r3, r4 = r, r, r, r
v1, v2, v3, v4 = v, v, v, v

for i in range(678):
    r1,v1 = propagate_noJ2(r1,v1, 10)
    r2,v2 = propagate_J2(r2,v2, 10)
    r3,v3 = propagate_J3(r3,v3, 10)
    r4,v4 = propagate_J2J3(r4,v4)
    
    X1.append(r1[0])
    Y1.append(r1[1])
    Z1.append(r1[2])


    X2.append(r2[0])
    Y2.append(r2[1])
    Z2.append(r2[2])

    X3.append(r3[0])
    Y3.append(r3[1])
    Z3.append(r3[2])

    X4.append(r4[0])
    Y4.append(r4[1])
    Z4.append(r4[2])

In [9]:
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
X4 = np.array(X4)

Y1 = np.array(Y1)
Y2 = np.array(Y2)
Y3 = np.array(Y3)
Y4 = np.array(Y4)

Z1 = np.array(Z1)
Z2 = np.array(Z2)
Z3 = np.array(Z3)
Z4 = np.array(Z4)

In [14]:
plt.figure(dpi=250)
plt.plot(X1-real_X, 'ob',     linewidth=0.7,markersize=3, markevery=15)
plt.plot(X2-real_X, '-dr',     linewidth=0.7,markersize=3, markevery=15)
plt.plot(X3-real_X, ':k',     linewidth=0.7)
plt.plot(X4-real_X, 'og',     linewidth=0.7,markersize=3, markevery=15)
plt.ylabel('km')
plt.legend(['Two-body vs HPOP','J2 vs HPOP','J3 vs HPOP','J2J3 vs HPOP',])
plt.title('X - LCLF')
plt.show()

In [15]:
plt.figure(dpi=250)
plt.plot(Y1-real_Y, 'ob',     linewidth=0.7,markersize=3, markevery=15)
plt.plot(Y2-real_Y, '-dr',    linewidth=0.7,markersize=3, markevery=15)
plt.plot(Y3-real_Y, ':k',     linewidth=0.7)
plt.plot(Y4-real_Y, 'og',     linewidth=0.7,markersize=3, markevery=15)
plt.ylabel('km')
plt.legend(['Two-body vs HPOP','J2 vs HPOP','J3 vs HPOP','J2J3 vs HPOP',])
plt.title('Y - LCLF')
plt.show()

In [16]:
plt.figure(dpi=250)
plt.plot(Z1-real_Z, 'ob',     linewidth=0.7,markersize=3, markevery=15)
plt.plot(Z2-real_Z, '-dr',    linewidth=0.7,markersize=3, markevery=15)
plt.plot(Z3-real_Z, ':k',     linewidth=0.7)
plt.plot(Z4-real_Z, 'og',     linewidth=0.7,markersize=3, markevery=15)
plt.ylabel('km')
plt.legend(['Two-body vs HPOP','J2 vs HPOP','J3 vs HPOP','J2J3 vs HPOP',])
plt.title('Z - LCLF')
plt.show()